In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation , Dropout
from tensorflow.keras.utils import to_categorical 
from sklearn.metrics import classification_report 

In [ ]:

data_train = pd.read_csv('/clinicalData-train.csv')
data_train = data_train.replace("?", np.nan)
data_test = pd.read_csv('/clinicalData-test.csv')
data_test = data_test.replace("?", np.nan)


In [ ]:
## ortalamaya gore numerical sutuna atama
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
missing_table = data_train.iloc[:,[0,17,19,21,23,25]].values 
imputer = imputer.fit(missing_table[:,:]) 
missing_table[:,:] = imputer.transform(missing_table[:,:]) 

tahmin_edilecek = data_train.iloc[:,-1].values



In [ ]:
## min-max normalizasyon
min_max_scaler = preprocessing.MinMaxScaler()
missing_table_minmax = min_max_scaler.fit_transform(missing_table)


In [ ]:
## en cok tekrar eden veriyi str sutuna atama
imputer_str = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
missing_table2 = data_train.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,22,24,26,27]].values
imputer_str = imputer_str.fit(missing_table2)
missing_table2 = imputer_str.transform(missing_table2)

In [ ]:
#encoding
le = LabelEncoder()
tahmin_edilecek = le.fit_transform(tahmin_edilecek)
i=0
for i in range(22):
    missing_table2[:,i] = le.fit_transform(missing_table2[:,i])

ohe = OneHotEncoder()
tahmin_edilecek = ohe.fit_transform(tahmin_edilecek.reshape(-1,1)).toarray()


In [ ]:
##test datasi veri isleme
test_missing_table = data_test.iloc[:,[0,17,19,21,23,25]].values 
imputer = imputer.fit(test_missing_table[:,:])
test_missing_table[:,:] = imputer.transform(test_missing_table[:,:])

test_missing_table2 = data_test.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,22,24,26,27]].values
imputer_str = imputer_str.fit(test_missing_table2)
test_missing_table2 = imputer_str.transform(test_missing_table2)

min_max_scaler = preprocessing.MinMaxScaler()
test_missing_table_minmax = min_max_scaler.fit_transform(test_missing_table)

i=0
for i in range(22):
    test_missing_table2[:,i] = le.fit_transform(test_missing_table2[:,i])
   
test_tahmin_edilecek = data_test.iloc[:,-1]
test_tahmin_edilecek = le.fit_transform(test_tahmin_edilecek)

test_tahmin_edilecek = ohe.fit_transform(test_tahmin_edilecek.reshape(-1,1)).toarray()

In [ ]:
##veri birleştirme
sonuc = pd.DataFrame(data = missing_table2,index= range(3016),columns = ["Sex","On thyroxine","query on thyroxine",
                                                                      "on antithyroid medication","sick","pregnant","thyroid surgery","I131 treatment","query hypothyroid",
                                                                      "query hyperthyroid","lithium","goitre","tumor","hypopituitary","psych","TSH measured","T3 measured",
                                                                      "TT4 measured","T4U measured","FTI measured","TBG measured","refferal source"]) 
sonuc2 = pd.DataFrame(data = missing_table_minmax,index= range(3016),columns = ["Age","TSH","T3","TT4","T4U","FTI"])
sonuc3 = pd.DataFrame(data = tahmin_edilecek,index = range(3016), columns = ['compensated_hypothyroid','negative','primary_hypotroid'])

s = pd.concat([sonuc,sonuc2], axis= 1)
df = pd.concat([s,sonuc3], axis=1)

test_sonuc = pd.DataFrame(data = test_missing_table2,index= range(754),columns = ["Sex","On thyroxine","query on thyroxine",
                                                                      "on antithyroid medication","sick","pregnant","thyroid surgery","I131 treatment","query hypothyroid",
                                                                      "query hyperthyroid","lithium","goitre","tumor","hypopituitary","psych","TSH measured","T3 measured",
                                                                      "TT4 measured","T4U measured","FTI measured","TBG measured","refferal source"]) 
test_sonuc2 = pd.DataFrame(data = test_missing_table_minmax,index= range(754),columns = ["Age","TSH","T3","TT4","T4U","FTI"])
test_sonuc3 = pd.DataFrame(data = test_tahmin_edilecek,index = range(754), columns = ['compensated_hypothyroid','negative','primary_hypotroid'])

s2 = pd.concat([test_sonuc,test_sonuc2], axis= 1)
df_test = pd.concat([s2,test_sonuc3], axis=1)


In [ ]:
y_train = df[['compensated_hypothyroid','negative','primary_hypotroid']]
x_train = df.drop(['compensated_hypothyroid','negative','primary_hypotroid'],axis = 1).values

y_test = df_test[['compensated_hypothyroid','negative','primary_hypotroid']]
x_test = df_test.drop(['compensated_hypothyroid','negative','primary_hypotroid'], axis = 1).values


x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

## model ve katman olusturma
model = Sequential()
model.add(Dense(16,kernel_initializer="uniform",activation="relu",input_dim=x_train.shape[1]))
model.add(Dense(14,kernel_initializer="uniform",activation="relu"))
model.add(Dense(12,kernel_initializer="uniform",activation="relu"))
model.add(Dense(6,kernel_initializer="uniform",activation="relu"))
model.add(Dense(3,kernel_initializer="uniform",activation="sigmoid")) ##cikis katmanı
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=['accuracy']) ##model compilesi
model.fit(x_train,y_train,epochs=50,batch_size=24) ## verileri koyma

In [ ]:
#test datasina gecis etc.
from sklearn import metrics
tahmin_birlestirme = np.argmax(model.predict(x_test),axis=1) ## 3 colonu tek kolona indirgiyor ve en yuksek tahmini belirliyoruz
y_test_birlestirme = np.argmax(y_test.values,axis=1) ### hangi y degerine ait oldugunu tespit ediyoruz
y_train_birlestirme = np.argmax(y_train.values,axis=1) 
tahmin_birlestirme_train = np.argmax(model.predict(x_train),axis=1)

In [ ]:
## precision f1 recall degerlerini cekme
target_names = ['compensated_hypothyroid','negative','primary_hypotroid']
print(classification_report(y_test_birlestirme, tahmin_birlestirme,target_names=target_names,zero_division=0, output_dict=False,digits=4))
print(classification_report(y_train_birlestirme, tahmin_birlestirme_train,target_names=target_names,zero_division=0, output_dict=False,digits=4))